# Example for running the default LCA model in AeroMAPS

The `LifeCycleAssessmentDefault` module enables to assess the life-cycle impacts of scenarios with no specific requirements or customizations. It relies on a pre-compiled LCA model stored in a JSON file. This example notebook demonstrates how to set up and run an AeroMAPS simulation using the default LCA model, and visualize the results.

> Note: while convenient, the default LCA model may not cover all specific needs. For example, the default LCA model cannot deal with custom energy carriers or specific processes added by the user. In such cas
> es, users are encouraged to develop custom LCA models tailored to their scenarios and requirements, as described in the [custom LCA example notebook](./life_cycle_assessment_custom.ipynb).

## Import librairies and configuration

Let's start by importing the necessary libraries and models, including the `LifeCycleAssessmentDefault` model.

In [ ]:
# --- Import libraries ---
%matplotlib widget
import sys
sys.path.insert(0, './utils/')
import pandas as pd
import brightway2 as bw
import lca_algebraic as agb
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import math
import collections
import time
from plots import plot_stacked_evolution_subplots
plt.style.use("bmh")

# --- Aeromaps models and processes ---
from aeromaps import create_process
from aeromaps.core.models import (
    models_traffic,
    models_efficiency_top_down_interp,
    models_energy_with_fuel_effect,
    models_offset,
    models_climate_fair,
    models_energy_cost,
    models_operation_cost_top_down,
    models_abatements_cost_simplified,
)
from aeromaps.models.impacts.life_cycle_assessment.life_cycle_assessment_default import LifeCycleAssessmentDefault

## Set models

The `LifeCycleAssessmentDefault` model takes two inputs: a `.json` file containing the pre-compiled functions, and the optional argument `split_by` which enables to split the impacts to get the individual contributions of the air transport components (aircraft production, fuel combustion, etc.). You may have a look at the [lca_model.json](./data/lca_data/precompiled_model/lca_model.json) file to check the different impact categories and LCA parameters for this default LCA model.

In [ ]:
# --- Set AeroMAPS models for this simulation ---
models = {
    "models_traffic": models_traffic,
    "models_efficiency_top_down_interp": models_efficiency_top_down_interp,
    "models_energy_with_fuel_effect": models_energy_with_fuel_effect,
    "models_offset": models_offset,
    "models_climate_fair": models_climate_fair,
    "models_energy_cost": models_energy_cost,
    "models_operation_cost_top_down": models_operation_cost_top_down,
    "models_abatements_cost_simplified": models_abatements_cost_simplified,
    "life_cycle_assessment": LifeCycleAssessmentDefault(
        json_file="./data/lca_data/precompiled_model/lca_model.json", split_by="phase"
    ),
}

## Create process

This example notebook covers the ICAO's LTAG integrated scenarios. The corresponding configuration files are provided in the `./data/config_files` directory.

In [ ]:
# --- Create process for IS scenario ---
process = create_process(
    configuration_file="data/config_files/config_is2medium.json",  # <-- select LTAG IS scenario (from 0 to 3)
    models=models,
)

## Run AeroMAPS simulation and assessment

You may now run AeroMAPS and visualize the results.

In [ ]:
# --- Run assessment ---
start_time = time.time()
process.compute()
process.write_json()
print("--- %s seconds ---" % (time.time() - start_time))

## Visualize LCA results

In [ ]:
# --- Results ---
process_data_vector_outputs = process.data["vector_outputs"]
process_data_float_inputs = process.data["float_inputs"]
process_data_climate = process.data["climate_outputs"]
lca_outputs = process.data["lca_outputs"]
lca_outputs

In [ ]:
# --- Plots ---
plt.close()
plot_stacked_evolution_subplots(lca_outputs)